<a href="https://colab.research.google.com/github/xcellentbird/playground/blob/main/bert_imdb_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.8 MB/s eta 0:00:00


# Load Dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset('imdb')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
from datasets import concatenate_datasets

merged_dataset = concatenate_datasets([dataset['train'], dataset['test']])
dataset = merged_dataset.train_test_split(
    test_size=0.2,
    seed=42,
    shuffle=True,
    stratify_by_column='label'
)

In [4]:
import numpy as np

train_text_len_mean = np.mean([len(text) for text in dataset['train']['text']])
test_text_len_mean = np.mean([len(text) for text in dataset['test']['text']])

train_text_len_mean, test_text_len_mean

(np.float64(1306.594925), np.float64(1320.7754))

In [5]:
train_label_unique = np.unique(dataset['train']['label'], return_counts=True)
test_label_unique = np.unique(dataset['test']['label'], return_counts=True)

train_label_unique, test_label_unique

((array([0, 1]), array([20000, 20000])), (array([0, 1]), array([5000, 5000])))

# Load Model

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification

model_id = 'bert-base-uncased'

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    torch_dtype='auto',
    device_map='auto'
)
tokenizer = BertTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [9]:
model.loss_function

<function transformers.loss.loss_utils.ForSequenceClassificationLoss(labels: torch.Tensor, pooled_logits: torch.Tensor, config, **kwargs) -> torch.Tensor>

# Train

In [10]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True).to(model.device)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [11]:
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy='steps',
    eval_steps=20,
    save_steps=20,
    logging_steps=1,
    load_best_model_at_end = True,
    metric_for_best_model='accuracy',
    report_to='wandb',
    run_name='bert-imdb-0823-2'
)

In [13]:
from transformers import EarlyStoppingCallback

callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
    # callbacks=callbacks
)
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xcellentbird (xcellentbird-private) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy
20,0.552500,0.554206,0.730800
40,0.237300,0.293481,0.888200
60,0.145300,0.275234,0.895200
80,0.211400,0.294880,0.886700
100,0.253800,0.278062,0.891500
120,0.284500,0.250552,0.897600
140,0.271600,0.315793,0.887500
160,0.302300,0.237704,0.909100
180,0.338400,0.207814,0.920400
200,0.121800,0.206282,0.924400


TrainOutput(global_step=1875, training_loss=0.1399656515839199, metrics={'train_runtime': 8332.4058, 'train_samples_per_second': 14.402, 'train_steps_per_second': 0.225, 'total_flos': 3.15733266432e+16, 'train_loss': 0.1399656515839199, 'epoch': 3.0})

In [15]:
trainer.evaluate()

{'eval_loss': 0.1816074103116989,
 'eval_accuracy': 0.9456,
 'eval_runtime': 63.7881,
 'eval_samples_per_second': 156.769,
 'eval_steps_per_second': 2.461,
 'epoch': 3.0}

In [16]:
from datasets import Dataset
from torch.nn.functional import softmax
from torch import tensor

label_names = dataset['train'].features['label'].names

def predict(text: str) -> str:
    ds = Dataset.from_dict({"text": [text]})
    tokenized_ds = ds.map(tokenize_function, batched=True)

    pred = trainer.predict(tokenized_ds).predictions
    probs = softmax(tensor(pred), dim=-1)
    pred_ids = probs.argmax(axis=-1)

    return label_names[pred_ids]

In [17]:
predict('hi, go')

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

'pos'